In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

from collections import Counter
import string
import re
import argparse
import json
import sys

In [2]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [3]:
def identify_question_type(question):
    if 'what' in question:
        return 'what'
    elif 'who' in question:
        return 'who'
    elif 'how' in question:
        return 'how'
    elif 'when' in question:
        return 'when'
    elif 'which' in question:
        return 'which'
    elif 'where' in question:
        return 'where'
    elif 'why' in question:
        return 'why'
    else:
        return 'other'

In [4]:
with open('../predictions.json') as f:
    predictions = json.load(f)
    
with open('../data/dev-v1.1.json') as f:
    dataset_json = json.load(f)
    dataset = dataset_json['data']

In [5]:
question_type_list = []
context_len_list = []
query_len_list = []
avg_answer_len_list = []
tmp_exact_match_list = []
tmp_f1_list = []

err_context = []
err_question = []
err_ground_truth = []
err_prediction = []
err_em = []


f1 = exact_match = total = 0
for article in dataset:
    for paragraph in article['paragraphs']:
        for qa in paragraph['qas']:
            total += 1
            if qa['id'] not in predictions:
                message = 'Unanswered question ' + qa['id'] + \
                          ' will receive score 0.'
                print(message, file=sys.stderr)
                continue
            ground_truths = list(map(lambda x: x['text'], qa['answers']))
            prediction = predictions[qa['id']]
            question = qa['question']
            
            question_type = identify_question_type(normalize_answer(question))
            context_len = len(normalize_answer(paragraph['context']).split())
            query_len = len(normalize_answer(qa['question']).split())
            avg_answer_len = sum([len(normalize_answer(item).split()) for item in ground_truths]) / len(ground_truths)
            
            tmp_exact_match = metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            tmp_f1 = metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)
            
            question_type_list.append(question_type)
            context_len_list.append(context_len)
            query_len_list.append(query_len)
            avg_answer_len_list.append(avg_answer_len)
            tmp_exact_match_list.append(tmp_exact_match)
            tmp_f1_list.append(tmp_f1)
            
            err_context.append(paragraph['context'])
            err_question.append(question)
            err_ground_truth.append(ground_truths)
            err_prediction.append(prediction)
            err_em.append(tmp_exact_match)
            
            exact_match += metric_max_over_ground_truths(
                exact_match_score, prediction, ground_truths)
            f1 += metric_max_over_ground_truths(
                f1_score, prediction, ground_truths)

exact_match = 100.0 * exact_match / total
f1 = 100.0 * f1 / total

In [6]:
df = pd.DataFrame({'context': err_context, 'question': err_question, 
                 'ground_truths': err_ground_truth, 'prediction': err_prediction, 
                 'em': err_em})

In [7]:
df = df[df['em'] == False].reset_index(drop=True)

In [8]:
df.shape

(4336, 5)

In [9]:
np.random.seed(27)
df = df.ix[np.random.choice(4336, 50)]

/home/zhengyang-pc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [10]:
df.head()

,context,em,ground_truths,prediction,question
3768,The zeta function is closely related to prime ...,False,"[identity, algebraic, modern algebraic number ...",modern algebraic,Of what mathematical nature is the Basel problem?
3912,"For the next three hundred years, Scotland was...",False,"[directly elected Scottish Assembly, directly ...",a directly elected scottish assembly to legisl...,What did Kllbrandon's report in 1973 recommend...
4237,Johnson's expedition was better organized than...,False,"[inconclusively, with both sides withdrawing f...",fort edward and fort william henry,Who won the battle of Lake George?
2929,"In many parts of the United States, after the ...",False,"[segregation academies, segregation academies]",shut down,What is a derogatory term for the Christian ac...
1026,"Johann Eck, speaking on behalf of the Empire a...",False,"[stood by their contents, whether he stood by ...",copies of his writings laid out on a table,"After asking if the books were his, what else ..."


In [17]:
for step, row in enumerate(df.iterrows()):
    print('Step: ')
    print(step+2)
    print('\n')
    temp = row[1]
    print('Context: ')
    print(temp['context'])
    print('\n')
    print('Question: ')
    print(temp['question'])
    print('\n')
    print('Ground Truths: ')
    print(temp['ground_truths'])
    print('\n')
    print('Prediction: ')
    print(temp['prediction'])
    print('\n')


Step: 
2


Context: 
The zeta function is closely related to prime numbers. For example, the aforementioned fact that there are infinitely many primes can also be seen using the zeta function: if there were only finitely many primes then ζ(1) would have a finite value. However, the harmonic series 1 + 1/2 + 1/3 + 1/4 + ... diverges (i.e., exceeds any given number), so there must be infinitely many primes. Another example of the richness of the zeta function and a glimpse of modern algebraic number theory is the following identity (Basel problem), due to Euler,


Question: 
Of what mathematical nature is the Basel problem?


Ground Truths: 
['identity', 'algebraic', 'modern algebraic number theory', 'modern algebraic number theory']


Prediction: 
modern algebraic


Step: 
3


Context: 
For the next three hundred years, Scotland was directly governed by the Parliament of Great Britain and the subsequent Parliament of the United Kingdom, both seated at Westminster, and the lack of a Parl

In [28]:
manual_err_analysis = pd.read_excel('BCN_error_analysis.xlsx')

In [30]:
manual_err_analysis = manual_err_analysis.rename(columns={
    'imprecise': 'Imprecise answer boundaries',
    'syntactic': 'Syntactic complications and ambiguities',
    'paraphrase': 'Paraphrase problems',
    'mult--sentence': 'Multi-sentence',
    'padding sta': 'Padding strategy',
    'external': 'External knowledge'
})

In [31]:
manual_err_analysis = manual_err_analysis.fillna(0)

In [32]:
manual_err_analysis.to_excel('BCN_error_analysis.xlsx', index=False)

In [33]:
manual_err_analysis

,Imprecise answer boundaries,Syntactic complications and ambiguities,Paraphrase problems,Multi-sentence,Padding strategy,External knowledge
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
manual_err_analysis.sum()

Imprecise answer boundaries                23.0
Syntactic complications and ambiguities    15.0
Paraphrase problems                         2.0
Multi-sentence                              6.0
Padding strategy                            3.0
External knowledge                          1.0
dtype: float64

In [35]:
manual_err_analysis.sum() / 50

Imprecise answer boundaries                0.46
Syntactic complications and ambiguities    0.30
Paraphrase problems                        0.04
Multi-sentence                             0.12
Padding strategy                           0.06
External knowledge                         0.02
dtype: float64